Install environment

# ConnectX environment was defined in v0.1.6
!pip install 'kaggle-environments>=0.1.6'

Import libraries

In [ ]:
# Here we import standard libraries and our environment
# You must first add the data for the task in the settings column
import random
import numpy as np
from kaggle_environments import make, evaluate

# Create the game environment
env = make("connectx", debug=True)

# List the available agents
print(list(env.agents))

Two Random agents

In [ ]:
env.run(["random", "random"])

# To render using iPython, we have to use a notebook as the Kaggle editor can't show HTML objects
env.render(mode="ipython")

Create agents

In [ ]:
# Agent 1: Random
def agent_random(obs, config):
    import random
    valid_moves = [col for col in range(config.columns) if obs.board[col]==0]
    return random.choice(valid_moves)

# Agent 2: Middle
def agent_middle(obs, config):
    return config.columns//2

# Agent 3: Leftmost
def agent_leftmost(obs, config):
    valid_moves = [col for col in range(config.columns) if obs.board[col]==0]
    return valid_moves[0]

Play

In [ ]:
# Choosing the random and leftmost agents
env.run([agent_random, agent_leftmost])
env.render(mode="ipython")

Winning Percentages

In [ ]:
def get_win_percentages(agent1, agent2, n_rounds=100):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

A winning move

In [ ]:
# Gets board at next step if agent drops piece in selected column
def drop_piece(grid, col, piece, config):
    next_grid = grid.copy()
    for row in range(config.rows-1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = piece
    return next_grid

# Returns True if dropping piece in column results in game win
def check_winning_move(obs, config, col, piece):
    import numpy as np
    
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    next_grid = drop_piece(grid, col, piece, config)
    # horizontal
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(next_grid[row,col:col+config.inarow])
            if window.count(piece) == config.inarow:
                return True
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(next_grid[row:row+config.inarow,col])
            if window.count(piece) == config.inarow:
                return True
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(next_grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if window.count(piece) == config.inarow:
                return True
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(next_grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if window.count(piece) == config.inarow:
                return True
    return False


A smarter agent

In [ ]:
def agent_blocker(obs, config):
    # This function will be our submission so must be self-contained
    import random
    
    # Produce a list of valid moves
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    
    # If we have a winning move, take it
    for col in valid_moves:
        if check_winning_move(obs, config, col, obs.mark):
            return col 
    
    # Else if the opponent has a winning move, block it
    opponent = (obs.mark%2)+1
    for col in valid_moves:
        if check_winning_move(obs, config, col, opponent):
            return col
    
    # Else return a random move
    return random.choice(valid_moves)


In [ ]:
# Run once to observe the procedure is implemented correctly
env.run([agent_blocker, agent_random])
env.render(mode="ipython")

# Determine the winning percentages with
get_win_percentages(agent1=agent_blocker, agent2=agent_random)

Function to write a file

In [ ]:
import inspect
import os

f = open("submission.py", "w")
f.write(inspect.getsource(agent_leftmost))
f.close()

print("agent_leftmost", "written to", "submission.py")